In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms, models
import torch.optim as optim
import os
import random
import cv2
import numpy as np

In [2]:
class_label_live = 1
class_label_print_attack_1 = 2
class_label_print_attack_2 = 3
class_label_display_attack_1 = 4
class_label_display_attack_2 = 5 

OULU-NPU Dataset Protocol I:  
Training -> 1200 (Real 240, Attack 960) \
Validation -> 900 (Real 180, Attack 720) \
Testing -> 600 (Real 120, Attack 480)

OULU-NPU Dataset Protocol II:  
Training -> 1080 (Real 360, Attack 720) \
Validation -> 810 (Real 270, Attack 540) \
Testing -> 1080 (Real 360, Attack 720)

In [3]:
data_path_test_live = '/protocol_I/test/real'
data_path_test_print_attack_1 = '/protocol_I/test/attack_sep/print1'
data_path_test_print_attack_2 = '/protocol_I/test/attack_sep/print2'
data_path_test_display_attack_1 = '/protocol_I/test/attack_sep/display1'
data_path_test_display_attack_2 = '/protocol_I/test/attack_sep/display2'

In [4]:
def load_samples(path, class_label, transform): #Select N frames returned from read_all_frames and assign labels to all samples of same class
        frames = read_all_frames(path)
        total_frames = list(range(0, frames.shape[0], 1))
        selected_samples = 5
        selected_frame = total_frames[selected_samples]
        samples =[]
        # Assign the same class label to all samples
        label = class_label
        samples =(transform(frames[selected_frame].squeeze()), label)     
        return samples

def read_all_frames(video_path): # reads all frames from a particular video and converts them to PyTorch tensors.
    frame_list = []
    video = cv2.VideoCapture(video_path)
    success = True
    while success:
        success, frame = video.read()
        if success:
            frame = cv2.resize(frame, (256, 256), interpolation=cv2.INTER_AREA) #framesize kept 256 x 256 
            frame_list.append(frame)
    frame_list = np.array(frame_list)
    return frame_list

class VideoDataset(Dataset):
    def __init__(self, data_path, class_label):
        self.data_path = data_path #path for directory containing video files
        self.video_files = [file for file in os.listdir(data_path) if file.endswith('.avi')]
        self.class_label = class_label #manually assign class_label for your desired class while loading
        self.data_length = len(self.video_files) 
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __len__(self): # returns the total number of samples in the dataset
        return self.data_length

    def __getitem__(self, idx): # loads and returns a sample from the dataset at the given index
        file = self.video_files[idx]
        path = os.path.join(self.data_path, file)
        frames= load_samples(path, self.class_label, self.transform)

        return frames

In [5]:
test_dataset_live = VideoDataset(data_path_test_live, class_label_live)
test_dataset_print_attack_1 = VideoDataset(data_path_test_print_attack_1, class_label_print_attack_1)
test_dataset_print_attack_2 = VideoDataset(data_path_test_print_attack_2, class_label_print_attack_2)
test_dataset_display_attack_1 = VideoDataset(data_path_test_display_attack_1, class_label_display_attack_1)
test_dataset_display_attack_2 = VideoDataset(data_path_test_display_attack_2, class_label_display_attack_2)

In [6]:
concatenated_test_dataset = ConcatDataset([test_dataset_live, test_dataset_print_attack_1, test_dataset_print_attack_2, test_dataset_display_attack_1, test_dataset_display_attack_2])
concatenated_test_loader = DataLoader(concatenated_test_dataset, batch_size=64, shuffle=False, pin_memory=True, num_workers=8)

In [7]:
# Print dataset sizes
print(f"Test set size: {len(concatenated_test_dataset)}")

Test set size: 600


In [8]:
# Load pre-trained MobileNetV2
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(in_features=1280, out_features=2) #default in_features =1280, out_features = 1000
# print(model)

/home/taha/anaconda3/envs/pt_fpad/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/taha/anaconda3/envs/pt_fpad/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the path to your saved model file
model_path = '/OULUprotocols/protocol_I/OULUNPU_prot_I.pth' # before OULU_prot_I.pth after OULU_prot_I_FB.pth before OULU_prot_II.pth after OULU_prot_II_FB.pth 

# Load the saved model
checkpoint = torch.load(model_path)

# Load the model's state dictionary
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [10]:
# Evaluate on the test set
test_correct = 0
test_total = 0

model.eval()
with torch.no_grad():
    
    test_cat_labels = torch.empty(0, dtype=torch.int64, device=device)
    test_predicted_cat_labels = torch.empty(0, dtype=torch.int64, device=device)
    test_model_op_cat = torch.empty(0, dtype=torch.int64, device=device)

    for test_images, test_labels in concatenated_test_loader:
        test_images, test_labels = test_images.to(device), test_labels.to(device)
        test_model_op = model(test_images)
        _, test_predicted = torch.max(test_model_op, 1)
        test_correct += (test_predicted == test_labels).sum().item() 
        test_total += test_labels.size(0)

        test_cat_labels = torch.cat((test_cat_labels, test_labels))
        test_predicted_cat_labels = torch.cat((test_predicted_cat_labels, test_predicted))
        test_model_op_cat = torch.cat((test_model_op_cat, test_model_op))

    test_accuracy = test_correct / test_total * 100  
    print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 1.00%


In [12]:
test_cat_labels_cpu = test_cat_labels.cpu()
test_predicted_cat_labels_cpu = test_predicted_cat_labels.cpu()

In [ ]:
import torch.nn.functional as F

softmax_output = F.softmax(test_model_op_cat, dim=1)

test_model_op_cat_second_column = softmax_output[:, 1]

test_model_op_cat_second_column_cpu = test_model_op_cat_second_column.cpu()

Subtracted because our labels are opposite as compared to oulumetrics

our labels:
 0 - live
 1 - attack

 oulumetics:
 1 - live
 0 - attack

In [20]:
subtracted = 1 - test_model_op_cat_second_column_cpu

print(subtracted)

tensor([9.9994e-01, 9.3744e-01, 9.9734e-01, 9.9999e-01, 9.9941e-01, 9.7976e-01,
        7.7406e-01, 9.9987e-01, 9.9987e-01, 1.0000e+00, 9.9999e-01, 8.3714e-02,
        9.0387e-01, 9.9916e-01, 9.9885e-01, 1.0000e+00, 9.0859e-01, 1.0000e+00,
        1.0000e+00, 8.9238e-01, 9.9990e-01, 9.9585e-01, 8.8911e-01, 1.0000e+00,
        9.9999e-01, 9.5694e-01, 9.9824e-01, 9.9997e-01, 9.9935e-01, 9.9996e-01,
        9.9921e-01, 9.9759e-01, 9.8865e-01, 9.9970e-01, 9.4609e-01, 1.0000e+00,
        9.9594e-01, 9.9948e-01, 9.9998e-01, 1.0000e+00, 5.5976e-01, 9.9994e-01,
        9.9997e-01, 1.0000e+00, 9.9963e-01, 9.9773e-01, 9.9891e-01, 9.9994e-01,
        7.5041e-01, 9.9938e-01, 6.4764e-01, 9.9934e-01, 7.6653e-01, 9.9527e-01,
        9.9850e-01, 9.9999e-01, 9.9550e-01, 1.0000e+00, 8.4706e-01, 9.9738e-01,
        9.0888e-01, 1.0000e+00, 9.8564e-01, 6.4391e-02, 9.9954e-01, 9.9419e-01,
        9.9990e-01, 5.4730e-02, 9.9998e-01, 9.9992e-01, 9.9994e-01, 9.9991e-01,
        9.9971e-01, 9.9999e-01, 9.9999e-

Softmax with Threshold

In [24]:
import oulumetrics

# returns the metrics APCER, BPCER and ACER
apcer, bpcer, acer = oulumetrics.calculate_metrics(test_cat_labels_cpu, subtracted, 1-0.5868009924888611)

print(apcer)
print(bpcer)
print(acer)

0.016666666666666666
0.05
0.03333333333333333


Argmax

In [25]:
import oulumetrics

apcer, bpcer, acer = oulumetrics.calculate_metrics(test_cat_labels_cpu, 1-test_predicted_cat_labels)
print(apcer)
print(bpcer)
print(acer)

0.016666666666666666
0.05
0.03333333333333333
